### 简单的Ping 测试脚本
主要用于测试服务器是否能够使用python程序ping通设备

In [2]:
import subprocess

# 测试的IP地址列表
test_ips = ["192.168.56.3","192.168.56.4","192.168.56.5"]

print("开始网络连通性测试...")

# 逐个ping测试IP地址
for ip in test_ips:
    command = ['ping', '-c', str(3), '-W', str(5), ip]
    
    try:
        result = subprocess.run(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        if result.returncode == 0:
            print(f"✅ {ip} 可达")
        else:
            print(f"❌ {ip} 不可达")
            if result.stderr.strip():
                print("错误信息:", result.stderr.strip())

    except Exception as e:
        print(f"⚠️ 执行 ping {ip} 时出错: {e}")

print("网络测试完成")

开始网络连通性测试...
✅ 192.168.56.3 可达
✅ 192.168.56.4 可达
✅ 192.168.56.5 可达
网络测试完成


### 简单的SSH测试脚本
主要用于测试服务器是否能够使用python程序通过SSH协议连接设备

In [3]:
import paramiko
import time

ip = "192.168.56.3"
user = "user"
passwd = "User@12345"

ssh_client = paramiko.SSHClient()  # 实例化SSHClient
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # 自动添加策略，保存服务器主机名和密钥信息
ssh_client.connect(hostname=ip, username=user, password=passwd)  # 连接SSH服务端，以用户名和密码进行认证

command = ssh_client.invoke_shell()  # 在客户端激活shell                        
command.send("dis version\n")  # 输入dis version命令

time.sleep(2)  # 线程推迟2S运行 = 休眠2S
output = command.recv(65535)  # 接收的最大字节数
print(output.decode("ascii"))  # 打印结果以ASCII输出

ssh_client.close()  # 关闭SSHClient

dis version

 ******************************************************************************
* Copyright (c) 2004-2021 New H3C Technologies Co., Ltd. All rights reserved.*
* Without the owner's prior written consent,                                 *
* no decompiling or reverse-engineering shall be allowed.                    *
******************************************************************************

 <H3C>dis version
H3C Comware Software, Version 7.1.064, Release 0427P22
Copyright (c) 2004-2021 New H3C Technologies Co., Ltd. All rights reserved.
H3C MSR36-20 uptime is 0 weeks, 0 days, 0 hours, 31 minutes
Last reboot reason: User reboot
Boot image: flash:/msr36-cmw710-boot-r0424p22.bin
Boot image version: 7.1.064, Release 0427P22
  Compiled Mar 16 2021 15:00:00
Boot image: flash:/msr36-cmw710-system-r0424p22.bin
Boot image version: 7.1.064, Release 0427P22
  Compiled Mar 16 2021 15:00:00

CPU ID: 0x2
384M bytes DDR3 SDRAM Memory
1024M bytes Flash Memory
PCB               Version:

### 简单的FTP脚本
主要用于测试服务器是否能够使用python程序通过FTP协议备份设备配置

In [35]:
import paramiko
import time
from datetime import datetime
import re

ip = "192.168.56.3"
user = "user"
passwd = "User@12345"

SFTP_IP = "192.168.56.2"
SFTP_Uname = "py-auto-dev"
SFTP_pwd = "H3C-py"

def detect_output(output):
    # 判断顺序很重要，避免子串匹配错误
    if "The server is not authenticated. Continue? [Y/N]:" in output:
        return 1
    elif "Do you want to save the server public key? [Y/N]:" in output:
        return 2
    elif SFTP_Uname+ "@" + SFTP_IP +"'s password:" in output:
        return 0
    else:
        return None  # 或者返回 -1 表示未匹配任何情况

ssh_client = paramiko.SSHClient()  # 实例化SSHClient
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # 自动添加策略，保存服务器主机名和密钥信息
ssh_client.connect(hostname=ip, username=user, password=passwd)  # 连接SSH服务端，以用户名和密码进行认证

command = ssh_client.invoke_shell()  # 在客户端激活shell
new_time = datetime.now()
# print(new_time)

# 获取设备名
command.send("\x1a") # 这里为键入命令ctrl+Z，来保证设备退到了用户视图
command.send("\n") 
time.sleep(2)
name_result = command.recv(65535)
name_result = name_result.decode("ascii")
# print(name_result)

# 构造新的文件名称
device_name = re.findall('.*?<(.*?)>.*?',name_result,re.S)[0]
new_file_name = device_name + "_%s_%s_%s_%s_%s_startup.cfg"%(new_time.year,new_time.month,new_time.day,new_time.hour,new_time.minute)
print(new_file_name)

command.send("sftp "+ SFTP_IP +"\n")
command.send(SFTP_Uname + "\n")
time.sleep(2)  # 线程推迟2S运行 = 休眠2S
FTPLogin_output = command.recv(65535)  # 接收的最大字节数
FTPLogin_output = FTPLogin_output.decode("ascii")
# print(type(FTPLogin_output))

while True:
    tag = detect_output(FTPLogin_output)
    if tag == 0:
        break
    else:
        command.send("y\n")
        
command.send(SFTP_pwd +"\n")
time.sleep(2)  # 线程推迟2S运行 = 休眠2S
PWD_output = command.recv(65535)  # 接收的最大字节数
PWD_output = PWD_output.decode("ascii")

if "sftp>" in PWD_output:
    command.send("put startup.cfg /home/py-auto-dev/AutoDevPro/Device_Config_Backup/"+ new_file_name +"\n")
else:
    print("sftp error")

command.send("quit\n")

time.sleep(2)  # 线程推迟2S运行 = 休眠2S
output = command.recv(65535)  # 接收的最大字节数
print(output.decode("ascii"))

# 关闭 SSHClient
ssh_client.close()


H3C_2025_7_28_16_35_startup.cfg
put startup.cfg /home/py-auto-dev/AutoDevPro/Device_Config_Backup/H3C_2025_7_28_16_35_startup.cfg
quit
Uploading startup.cfg to /home/py-auto-dev/AutoDevPro/Device_Config_Backup/H3C_2025_7_28_16_35_startup.cfg
startup.cfg                                   100% 2759     2.7KB/s   00:00    
sftp> quit
<H3C>


### xlsx表格处理
阅读表格并处理其中信息的简单编程脚本。

In [3]:
import pandas as pd
import os

# 定义文件路径
file_path = "/home/py-auto-dev/AutoDevPro/Config_Sheet/Automated_Configuration_Table_Template.xlsx"

def get_sheet_names(file_path):
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"错误：文件不存在！路径为：{file_path}")
        return None

    try:
        # 使用 pandas 的 ExcelFile 获取工作表名称
        excel_file = pd.ExcelFile(file_path)
        sheet_names = excel_file.sheet_names  # 返回所有 sheet 名称的列表
        return sheet_names
    except Exception as e:
        print(f"读取 Excel 文件时发生错误：{e}")
        return None

# 调用函数
sheet_list = get_sheet_names(file_path)

if sheet_list is not None:
    print("所有工作表名称：")
    print(sheet_list)

所有工作表名称：
['Init_Sheet', 'Port_IP_Sheet(Router)', 'Port_IP_Sheet(Switch)', 'IP_Route_Static_Sheet', 'OSPF_Init_Sheet', 'OSPF_Sheet']


In [6]:
from openpyxl import load_workbook

def read_init_sheet_to_list_of_dicts(file_path, sheet_name='Init_Sheet'):
    """
    读取Excel文件中指定工作表的数据，转换为列表嵌套字典的形式
    :param file_path: Excel文件路径
    :param sheet_name: 工作表名称，默认为 'Init_Sheet'
    :return: 包含字典的列表，每行对应一个字典
    """
    # 加载工作簿和工作表
    workbook = load_workbook(file_path)
    sheet = workbook[sheet_name]

    # 获取所有行数据
    rows = list(sheet.iter_rows(values_only=True))

    if not rows:
        print("工作表为空")
        return []

    # 第一行作为键（列名）
    headers = [cell.strip() if cell else "" for cell in rows[0]]

    # 构建结果列表
    result = []
    for row in rows[1:]:
        # 每一行构建一个字典，与列头对应
        row_dict = {headers[i]: row[i] for i in range(len(headers))}
        result.append(row_dict)

    return result

# 使用示例
if __name__ == "__main__":
    file_path = '/home/py-auto-dev/AutoDevPro/Config_Sheet/Automated_Configuration_Table_Template.xlsx'  # 替换为你的实际文件路径
    data = read_init_sheet_to_list_of_dicts(file_path)

    # 打印结果
    for item in data:
        print(item)

{'Device_Name': 'AutoConfig_R1', 'Manage_IP': '192.168.56.3/24', 'Manager_Name': 'user', 'Manager_Password': 'User@12345', 'Backup_choose': 1, 'FTP_Server': None}
{'Device_Name': 'AutoConfig_R2', 'Manage_IP': '192.168.56.4/24', 'Manager_Name': 'user', 'Manager_Password': 'User@12345', 'Backup_choose': 1, 'FTP_Server': None}
{'Device_Name': 'AutoConfig_R3', 'Manage_IP': '192.168.56.5/24', 'Manager_Name': 'user', 'Manager_Password': 'User@12345', 'Backup_choose': 1, 'FTP_Server': None}
{'Device_Name': 'Auto_Dev_Server', 'Manage_IP': '192.168.56.2/24', 'Manager_Name': 'py-auto-dev', 'Manager_Password': 'H3C-py', 'Backup_choose': None, 'FTP_Server': 1}
